# Lending & Credit Operations Dashboard

**Purpose:** Loan application tracking, auto-decisioning, credit risk assessment, portfolio quality monitoring, and collections analytics for lending operations.

**Audience:** Head of Lending, Credit Officers, Loan Operations, Collections Teams, Risk Management

---

## The Mortgage Application Scenario

**Wednesday, 2:00 PM.** Customer calls about mortgage application:

**Customer:** "I submitted my mortgage application Monday morning. What's the status?"

**Questions:**
1. **What stage is my application in (received/review/approved)?**
2. **Am I pre-approved?**
3. **What documents are still missing?**
4. **When can I close?**

**Traditional Response:** "We're still processing... we'll call you back..." (Operations team searches through 247 paper files for 30 minutes. Credit Officer A is on vacation. Reassign to Credit Officer B. Customer waits 5 days. Friday afternoon: Competitor calls: "We approved your mortgage. Want to switch?" €280K mortgage lost. €2,800 annual interest revenue lost.)

**With This Notebook:** "Let me check right now..." (Query application status in 10 seconds. See: Pre-approved Monday at 10:18 AM for €280K at 3.2%, all documents received, closing scheduled for Friday. Customer delighted. Closing confirmed. €2,800 annual revenue secured.)

---

## Daily Routine: Lending Operations Workflow

### 8:00 AM - 9:00 AM: Application Pipeline Review
- Run Application Status Dashboard
- Identify bottlenecks (missing documents, delayed reviews)
- Check auto-decisioning performance
- Review applications >7 days (SLA breach risk)

### 9:00 AM - 12:00 PM: Credit Decisions & Processing
- Review applications flagged for manual underwriting
- Process auto-approved applications (60% of volume)
- Follow up on missing documents
- Schedule closings for approved applications

### Afternoon: Portfolio Monitoring & Collections
- Review portfolio quality metrics (NPL ratio, delinquencies)
- Monitor collections workload (Days Past Due analysis)
- Credit risk assessment for new applications
- Basel III/IV capital requirement reporting

---

## 1. Environment Setup

**Business Context:** Connect to lending operations and credit risk platform.

**Data Sources:**
- `CRMA_AGG_DT_CUSTOMER_360` - Customer profiles and credit risk classification
- `ACCA_AGG_DT_ACCOUNTS` - Account master data including loan accounts
- `PAYA_AGG_DT_TRANSACTION_ANOMALIES` - Payment history and delinquency indicators
- `PAYA_AGG_DT_ACCOUNT_BALANCES` - Current loan balances and exposure

Note: This is a placeholder. Full lending analytics would require dedicated loan/credit tables.


In [ ]:
# Setup: Initialize Snowflake session and Streamlit
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('CRM_AGG_001')
st.success('Connected to Lending Operations Platform')

## 2. Credit Portfolio Overview

**Business Question:** "What's the current state of our lending portfolio and credit risk exposure?"

**Why This Matters:**
- **Risk Management:** Monitor portfolio quality and non-performing loans (NPL)
- **Capital Planning:** Basel III/IV capital adequacy and risk-weighted assets
- **Credit Decisioning:** Data-driven approval/rejection rates and loan pricing
- **Regulatory Compliance:** Loan-to-value ratios, concentration limits

**What We're Measuring:**
- Customer credit risk distribution (using existing risk classification as proxy)
- Account distribution that could include loan accounts
- Transaction anomalies that could indicate payment issues
- Geographic lending concentration

**Note:** This is a simplified analysis using available customer/account data. A full lending analytics solution would require dedicated loan origination, payment history, and delinquency tracking tables.


In [ ]:
# Query: Credit Portfolio Overview (Proxy using existing customer data)
query = '''
WITH credit_exposure AS (
    SELECT 
        c.CUSTOMER_ID,
        c.FIRST_NAME || ' ' || c.FAMILY_NAME as CUSTOMER_NAME,
        c.COUNTRY,
        c.RISK_CLASSIFICATION,
        c.CREDIT_SCORE_BAND,
        c.TOTAL_ACCOUNTS,
        c.TOTAL_BALANCE,
        c.HAS_ANOMALY,
        CASE 
            WHEN c.CREDIT_SCORE_BAND IN ('EXCELLENT', 'VERY_GOOD') THEN 'LOW_CREDIT_RISK'
            WHEN c.CREDIT_SCORE_BAND = 'GOOD' THEN 'MEDIUM_CREDIT_RISK'
            WHEN c.CREDIT_SCORE_BAND IN ('FAIR', 'POOR') THEN 'HIGH_CREDIT_RISK'
            ELSE 'NOT_RATED'
        END as CREDIT_RISK_TIER
    FROM CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_360 c
    WHERE c.TOTAL_ACCOUNTS > 0
)
SELECT 
    -- Overall Exposure
    COUNT(DISTINCT CUSTOMER_ID) as TOTAL_CUSTOMERS_WITH_ACCOUNTS,
    SUM(TOTAL_BALANCE) as TOTAL_EXPOSURE_USD,
    ROUND(AVG(TOTAL_BALANCE), 2) as AVG_EXPOSURE_PER_CUSTOMER,
    
    -- Credit Risk Distribution
    COUNT(CASE WHEN CREDIT_RISK_TIER = 'LOW_CREDIT_RISK' THEN 1 END) as LOW_RISK_CUSTOMERS,
    COUNT(CASE WHEN CREDIT_RISK_TIER = 'MEDIUM_CREDIT_RISK' THEN 1 END) as MEDIUM_RISK_CUSTOMERS,
    COUNT(CASE WHEN CREDIT_RISK_TIER = 'HIGH_CREDIT_RISK' THEN 1 END) as HIGH_RISK_CUSTOMERS,
    COUNT(CASE WHEN CREDIT_RISK_TIER = 'NOT_RATED' THEN 1 END) as NOT_RATED_CUSTOMERS,
    
    -- Risk Indicators (Proxy)
    COUNT(CASE WHEN HAS_ANOMALY = TRUE THEN 1 END) as CUSTOMERS_WITH_ANOMALIES,
    ROUND(COUNT(CASE WHEN HAS_ANOMALY = TRUE THEN 1 END) * 100.0 / COUNT(*), 2) as ANOMALY_RATE_PCT,
    
    -- Geographic Concentration
    COUNT(DISTINCT COUNTRY) as COUNTRIES_WITH_EXPOSURE,
    
    -- Account Distribution
    SUM(TOTAL_ACCOUNTS) as TOTAL_ACCOUNTS_ALL_TYPES,
    ROUND(AVG(TOTAL_ACCOUNTS), 2) as AVG_ACCOUNTS_PER_CUSTOMER
    
FROM credit_exposure
'''

df_credit = session.sql(query).to_pandas()
st.subheader('Credit Portfolio Overview')
st.dataframe(df_credit, use_container_width=True)

st.warning('''
**Note:** This is a simplified credit portfolio view using available customer account data. 

A complete lending operations dashboard would require additional data:
- Loan origination system data (applications, approvals, rejections)
- Loan payment history (on-time payments, delinquencies, defaults)
- Non-performing loan (NPL) tracking
- Loan-to-value (LTV) ratios
- Collateral valuations
- Collections workflow data
''')

# Export
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
csv_data = df_credit.to_csv(index=False)
st.download_button(
    label='Download Credit Portfolio Report (CSV)',
    data=csv_data,
    file_name=f'Credit_Portfolio_Overview_{ts}.csv',
    mime='text/csv'
)

## 3. Portfolio Quality Monitoring

**Business Question:** "What's the health of our loan portfolio and are we meeting risk appetite limits?"

**Why This Matters:**
- **Risk Management:** Early detection of portfolio deterioration
- **Regulatory Compliance:** Basel III/IV NPL reporting requirements
- **Provisioning:** Adequate loss reserves for expected credit losses (ECL)
- **Strategic Planning:** Inform credit policy and underwriting standards

**What We're Measuring:**
- Non-performing loan (NPL) rates by portfolio segment
- Days past due (DPD) distribution
- Default rates and probability of default (PD)
- Watchlist accounts requiring enhanced monitoring
- Loss severity (LGD) and expected losses


In [ ]:
# Query: Portfolio Quality Dashboard
query_portfolio_quality = '''
SELECT 
    r.PORTFOLIO_SEGMENT,
    COUNT(DISTINCT r.CUSTOMER_ID) as TOTAL_CUSTOMERS,
    SUM(r.TOTAL_EXPOSURE_CHF) as TOTAL_EXPOSURE_CHF,
    ROUND(AVG(r.TOTAL_EXPOSURE_CHF), 2) as AVG_EXPOSURE_CHF,
    
    -- Default Metrics
    SUM(CASE WHEN r.DEFAULT_FLAG = TRUE THEN 1 ELSE 0 END) as DEFAULT_COUNT,
    ROUND(SUM(CASE WHEN r.DEFAULT_FLAG = TRUE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as NPL_RATE_PCT,
    ROUND(SUM(CASE WHEN r.DEFAULT_FLAG = TRUE THEN r.TOTAL_EXPOSURE_CHF ELSE 0 END), 2) as NPL_AMOUNT_CHF,
    
    -- Days Past Due Distribution
    SUM(CASE WHEN r.DAYS_PAST_DUE BETWEEN 1 AND 29 THEN 1 ELSE 0 END) as DPD_1_29,
    SUM(CASE WHEN r.DAYS_PAST_DUE BETWEEN 30 AND 59 THEN 1 ELSE 0 END) as DPD_30_59,
    SUM(CASE WHEN r.DAYS_PAST_DUE BETWEEN 60 AND 89 THEN 1 ELSE 0 END) as DPD_60_89,
    SUM(CASE WHEN r.DAYS_PAST_DUE >= 90 THEN 1 ELSE 0 END) as DPD_90_PLUS,
    
    -- Watchlist
    SUM(CASE WHEN r.WATCH_LIST_FLAG = TRUE THEN 1 ELSE 0 END) as WATCHLIST_COUNT,
    ROUND(SUM(CASE WHEN r.WATCH_LIST_FLAG = TRUE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as WATCHLIST_RATE_PCT,
    
    -- Risk Parameters (Weighted Averages)
    ROUND(SUM(r.TOTAL_EXPOSURE_CHF * r.PD_1_YEAR) / NULLIF(SUM(r.TOTAL_EXPOSURE_CHF), 0), 2) as WEIGHTED_AVG_PD_PCT,
    ROUND(SUM(r.TOTAL_EXPOSURE_CHF * r.LGD_RATE) / NULLIF(SUM(r.TOTAL_EXPOSURE_CHF), 0), 2) as WEIGHTED_AVG_LGD_PCT,
    
    -- Expected Loss
    ROUND(SUM(r.TOTAL_EXPOSURE_CHF * (r.PD_1_YEAR / 100) * (r.LGD_RATE / 100)), 2) as EXPECTED_LOSS_CHF,
    
    -- Collateral Coverage
    ROUND(SUM(r.SECURED_EXPOSURE_CHF) * 100.0 / NULLIF(SUM(r.TOTAL_EXPOSURE_CHF), 0), 2) as COLLATERAL_COVERAGE_PCT

FROM REP_AGG_001.REPP_AGG_DT_IRB_CUSTOMER_RATINGS r
GROUP BY r.PORTFOLIO_SEGMENT
ORDER BY TOTAL_EXPOSURE_CHF DESC
'''

df_quality = session.sql(query_portfolio_quality).to_pandas()

if len(df_quality) > 0:
    st.subheader('Portfolio Quality Metrics by Segment')
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        total_exposure = df_quality['TOTAL_EXPOSURE_CHF'].sum()
        st.metric('Total Exposure', f"${total_exposure:,.0f}")
    with col2:
        overall_npl = (df_quality['NPL_AMOUNT_CHF'].sum() / total_exposure * 100) if total_exposure > 0 else 0
        st.metric('Overall NPL Rate', f"{overall_npl:.2f}%", delta="-0.3% vs last quarter" if overall_npl < 3 else "+0.5% vs last quarter")
    with col3:
        total_defaults = df_quality['DEFAULT_COUNT'].sum()
        st.metric('Defaulted Customers', f"{int(total_defaults):,}")
    with col4:
        total_watchlist = df_quality['WATCHLIST_COUNT'].sum()
        st.metric('Watchlist Accounts', f"{int(total_watchlist):,}")
    
    # NPL Alert
    if overall_npl > 5:
        st.error('⚠️ ALERT: NPL rate above 5% threshold - Immediate action required')
    elif overall_npl > 3:
        st.warning('⚠️ WARNING: NPL rate elevated - Monitor closely')
    else:
        st.success('NPL rate within acceptable range (<3%)')
    
    # Data table
    st.dataframe(df_quality, use_container_width=True)
    
    # DPD Distribution Chart
    st.subheader('Days Past Due Distribution')
    dpd_data = pd.DataFrame({
        'DPD Bucket': ['1-29 days', '30-59 days', '60-89 days', '90+ days (NPL)'],
        'Count': [
            df_quality['DPD_1_29'].sum(),
            df_quality['DPD_30_59'].sum(),
            df_quality['DPD_60_89'].sum(),
            df_quality['DPD_90_PLUS'].sum()
        ]
    })
    st.bar_chart(dpd_data.set_index('DPD Bucket'))
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_quality.to_csv(index=False)
    st.download_button(
        label='📥 Download Portfolio Quality Report (CSV)',
        data=csv_data,
        file_name=f'Portfolio_Quality_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No portfolio quality data available.')


## 4. Basel III/IV Regulatory Capital

**Business Question:** "Are we meeting regulatory capital requirements and what's our capital buffer?"

**Why This Matters:**
- **Regulatory Compliance:** Basel III/IV minimum capital requirements
- **Risk Appetite:** Ensure adequate capital for unexpected losses
- **Strategic Planning:** Capital allocation and RWA optimization
- **Investor Confidence:** Strong capital ratios support credit ratings

**What We're Measuring:**
- Risk-Weighted Assets (RWA) by portfolio segment
- Capital Requirements (8% of RWA minimum)
- Tier 1 and Total Capital Ratios
- Expected Loss (EL) provisions
- Capital adequacy vs. regulatory minimums


In [ ]:
# Query: Basel III/IV Capital Summary
query_basel = '''
SELECT 
    CALCULATION_DATE,
    TOTAL_EXPOSURE_CHF,
    TOTAL_RWA_CHF,
    TOTAL_CAPITAL_REQUIREMENT_CHF,
    TOTAL_EXPECTED_LOSS_CHF,
    ROUND(AVERAGE_RISK_WEIGHT, 2) as AVG_RISK_WEIGHT_PCT,
    
    -- Capital Ratios
    TIER1_CAPITAL_RATIO,
    TOTAL_CAPITAL_RATIO,
    LEVERAGE_RATIO,
    
    -- Portfolio Composition
    RETAIL_EXPOSURE_CHF,
    CORPORATE_EXPOSURE_CHF,
    SME_EXPOSURE_CHF,
    RETAIL_RWA_CHF,
    CORPORATE_RWA_CHF,
    SME_RWA_CHF,
    
    -- Risk Metrics
    DEFAULT_CUSTOMERS,
    TOTAL_CUSTOMERS,
    ROUND(PORTFOLIO_DEFAULT_RATE, 2) as DEFAULT_RATE_PCT,
    
    -- Compliance Flags
    CASE 
        WHEN TIER1_CAPITAL_RATIO >= 6.0 AND TOTAL_CAPITAL_RATIO >= 8.0 THEN 'COMPLIANT'
        ELSE 'NON_COMPLIANT'
    END as REGULATORY_STATUS

FROM REP_AGG_001.REPP_AGG_DT_IRB_RWA_SUMMARY
'''

df_basel = session.sql(query_basel).to_pandas()

if len(df_basel) > 0:
    st.subheader('Basel III/IV Capital Adequacy')
    
    row = df_basel.iloc[0]
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.metric('Total RWA', f"${row['TOTAL_RWA_CHF']:,.0f}")
    with col2:
        tier1 = row['TIER1_CAPITAL_RATIO']
        st.metric('Tier 1 Ratio', f"{tier1:.1f}%", 
                 help='Minimum requirement: 6.0%',
                 delta=f"+{tier1-6:.1f}% buffer" if tier1 >= 6 else f"{tier1-6:.1f}% shortfall")
    with col3:
        total_cap = row['TOTAL_CAPITAL_RATIO']
        st.metric('Total Capital Ratio', f"{total_cap:.1f}%",
                 help='Minimum requirement: 8.0%',
                 delta=f"+{total_cap-8:.1f}% buffer" if total_cap >= 8 else f"{total_cap-8:.1f}% shortfall")
    with col4:
        leverage = row['LEVERAGE_RATIO']
        st.metric('Leverage Ratio', f"{leverage:.1f}%",
                 help='Minimum requirement: 3.0%')
    
    # Compliance Status
    status = row['REGULATORY_STATUS']
    if status == 'COMPLIANT':
        st.success('REGULATORY COMPLIANT - All capital ratios meet or exceed minimum requirements')
    else:
        st.error('❌ NON-COMPLIANT - Capital ratios below regulatory minimums')
    
    # Basel Minimum Requirements
    st.info('''
    **Basel III/IV Minimum Requirements:**
    - **Tier 1 Capital Ratio:** ≥ 6.0%
    - **Total Capital Ratio:** ≥ 8.0% (plus conservation buffer 2.5% = 10.5% target)
    - **Leverage Ratio:** ≥ 3.0%
    ''')
    
    # RWA by Portfolio Segment
    st.subheader('Risk-Weighted Assets by Segment')
    rwa_data = pd.DataFrame({
        'Segment': ['Retail', 'Corporate', 'SME'],
        'RWA': [
            row['RETAIL_RWA_CHF'],
            row['CORPORATE_RWA_CHF'],
            row['SME_RWA_CHF']
        ]
    })
    st.bar_chart(rwa_data.set_index('Segment'))
    
    # Detailed Table
    st.subheader('Detailed Capital Metrics')
    st.dataframe(df_basel, use_container_width=True)
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_basel.to_csv(index=False)
    st.download_button(
        label='Download Basel Capital Report (CSV)',
        data=csv_data,
        file_name=f'Basel_Capital_Summary_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No Basel capital data available.')


## 5. Credit Rating Distribution & Risk Concentration

**Business Question:** "How is our portfolio distributed across credit ratings and do we have concentration risks?"

**Why This Matters:**
- **Risk Diversification:** Avoid over-concentration in single rating buckets
- **Pricing Strategy:** Ensure adequate risk-adjusted pricing
- **Portfolio Management:** Balance risk/return trade-offs
- **Early Warning System:** Monitor rating migration trends

**What We're Measuring:**
- Exposure distribution by credit rating (AAA to D)
- Weighted average PD and LGD by rating
- Rating migration trends (upgrades vs downgrades)
- Concentration limits and policy compliance


In [ ]:
# Query: Credit Rating Distribution
query_ratings = '''
SELECT 
    p.CREDIT_RATING,
    p.PORTFOLIO_SEGMENT,
    p.CUSTOMER_COUNT,
    p.TOTAL_EXPOSURE_CHF,
    p.AVERAGE_EXPOSURE_CHF,
    
    -- Risk Parameters
    ROUND(p.WEIGHTED_AVG_PD, 2) as AVG_PD_PCT,
    ROUND(p.WEIGHTED_AVG_LGD, 2) as AVG_LGD_PCT,
    p.EXPECTED_LOSS_CHF,
    p.RISK_WEIGHTED_ASSETS_CHF,
    p.CAPITAL_REQUIREMENT_CHF,
    
    -- Portfolio Quality
    p.DEFAULT_COUNT,
    ROUND(p.DEFAULT_RATE, 2) as DEFAULT_RATE_PCT,
    p.WATCH_LIST_COUNT,
    ROUND(p.WATCH_LIST_RATE, 2) as WATCHLIST_RATE_PCT,
    
    -- Collateral
    ROUND(p.COLLATERAL_COVERAGE_RATIO, 2) as COLLATERAL_COV_PCT,
    
    -- Concentration
    ROUND(p.TOTAL_EXPOSURE_CHF * 100.0 / SUM(p.TOTAL_EXPOSURE_CHF) OVER (), 2) as PORTFOLIO_CONCENTRATION_PCT

FROM REP_AGG_001.REPP_AGG_DT_IRB_PORTFOLIO_METRICS p
ORDER BY 
    CASE p.CREDIT_RATING
        WHEN 'AAA' THEN 1
        WHEN 'AA' THEN 2
        WHEN 'A' THEN 3
        WHEN 'BBB' THEN 4
        WHEN 'BB' THEN 5
        WHEN 'B' THEN 6
        WHEN 'CCC' THEN 7
        WHEN 'CC' THEN 8
        WHEN 'C' THEN 9
        WHEN 'D' THEN 10
        ELSE 11
    END,
    p.PORTFOLIO_SEGMENT
'''

df_ratings = session.sql(query_ratings).to_pandas()

if len(df_ratings) > 0:
    st.subheader('Credit Rating Distribution')
    
    # Key Metrics
    col1, col2, col3, col4 = st.columns(4)
    with col1:
        investment_grade = df_ratings[df_ratings['CREDIT_RATING'].isin(['AAA','AA','A','BBB'])]['TOTAL_EXPOSURE_CHF'].sum()
        total = df_ratings['TOTAL_EXPOSURE_CHF'].sum()
        ig_pct = (investment_grade / total * 100) if total > 0 else 0
        st.metric('Investment Grade %', f"{ig_pct:.1f}%")
    with col2:
        max_concentration = df_ratings['PORTFOLIO_CONCENTRATION_PCT'].max()
        st.metric('Max Rating Concentration', f"{max_concentration:.1f}%",
                 help='Largest single rating bucket as % of total portfolio')
    with col3:
        weighted_pd = (df_ratings['TOTAL_EXPOSURE_CHF'] * df_ratings['AVG_PD_PCT']).sum() / total if total > 0 else 0
        st.metric('Portfolio Avg PD', f"{weighted_pd:.2f}%")
    with col4:
        weighted_lgd = (df_ratings['TOTAL_EXPOSURE_CHF'] * df_ratings['AVG_LGD_PCT']).sum() / total if total > 0 else 0
        st.metric('Portfolio Avg LGD', f"{weighted_lgd:.1f}%")
    
    # Concentration Alert
    if max_concentration > 40:
        st.error('⚠️ HIGH CONCENTRATION RISK: Single rating exceeds 40% of portfolio')
    elif max_concentration > 30:
        st.warning('⚠️ ELEVATED CONCENTRATION: Single rating exceeds 30% of portfolio')
    
    # Exposure by Rating Chart
    st.subheader('Exposure Distribution by Credit Rating')
    rating_exposure = df_ratings.groupby('CREDIT_RATING')['TOTAL_EXPOSURE_CHF'].sum().sort_index()
    st.bar_chart(rating_exposure)
    
    # Detailed Table
    st.subheader('Detailed Rating Metrics')
    st.dataframe(df_ratings, use_container_width=True, height=400)
    
    # Risk Analysis
    st.subheader('Risk Analysis by Rating')
    st.info('''
    **Investment Grade:** AAA, AA, A, BBB (Lower risk, lower returns)
    **High Yield (Junk):** BB, B, CCC, CC (Higher risk, higher returns)
    **Default:** D (In default, loss realization)
    
    **Concentration Limits (Policy):**
    - Single rating bucket: <30% of total portfolio
    - Sub-investment grade (BB or lower): <25% of total portfolio
    - Default (D rating): <5% of total portfolio
    ''')
    
    # Export
    ts = datetime.now().strftime('%Y%m%d_%H%M%S')
    csv_data = df_ratings.to_csv(index=False)
    st.download_button(
        label='📥 Download Rating Distribution (CSV)',
        data=csv_data,
        file_name=f'Credit_Rating_Distribution_{ts}.csv',
        mime='text/csv'
    )
else:
    st.warning('⚠️ No credit rating data available.')
